    Importing Libraries

In [153]:
import numpy as np
import pandas as pd
import yfinance as yf

    Data Exploration

In [154]:
df = yf.download("^NSEI", start = "2024-01-01", end = "2024-01-25")

def pct_change(arr):
    l = len(arr)
    res = arr.copy()
    res[0] = -1
    for i in range(1, l):
        res[i] = (arr[i]/arr[i-1] - 1)*100
    
    return res

def pct_change_cross(arr1, arr2):
    l = len(arr1)
    res = arr1.copy()
    res[0] = -1
    for i in range(1, l):
        res[i] = (arr1[i]/arr2[i-1] - 1)*100
    
    return res

df['Close to Close'] = pct_change(np.array(df['Close']))
df['Close to Open'] = pct_change_cross(np.array(df['Open']), np.array(df['Close']))
df['Close to Next High'] = pct_change_cross(np.array(df['High']), np.array(df['Close']))
df['Close to Next Low'] = pct_change_cross(np.array(df['Low']), np.array(df['Close']))

[*********************100%%**********************]  1 of 1 completed


    Calculating the Final Analysis of Profit and Loss Days

In [155]:
df_interest = pd.DataFrame()

high_positive = []
for i in range(len(np.array(df['Close to Next Low']))):
    if np.array(df['Close to Next High'])[i] > 0:
        high_positive.append(1)
    else:
        high_positive.append(0)

low_negative = []
for i in range(len(np.array(df['Close to Next Low']))):
    if np.array(df['Close to Next Low'])[i] < 0:
        low_negative.append(1)
    else:
        low_negative.append(0)

high_less_one = []
sl = 0
for i in range(len(np.array(df['Close to Next Low']))):
    if np.array(df['Close to Next High'])[i] < 1:
        high_less_one.append(1)
    else:
        high_less_one.append(0)
        sl += 1
        

low_less_one = []
for i in range(len(np.array(df['Close to Next Low']))):
    if np.array(df['Close to Next Low'])[i] > -1:
        low_less_one.append(1)
    else:
        low_less_one.append(0)
        sl += 1

res = []
perf = 0
for i in range(len(np.array(df['Close to Next Low']))):
    var = high_positive[i] and low_negative[i] and high_less_one[i] and low_less_one[i]
    res.append(var)
    if var == 1:
        perf += 1
        
df_interest['High Positive'] = high_positive
df_interest['Low Negative'] = low_negative
df_interest['High < 1%'] = high_less_one
df_interest['Low < 1%'] = low_less_one
df_interest['Result'] = res


print("REPORT ON LOSS")
print(" ")

print("Total SL days:", sl, "/", len(np.array(df['Close to Next Low'])), "\n", 
      "% Day SL hit", (sl/len(np.array(df['Close to Next Low'])))*100, "%")

print("--------------------------------------------------------------------------")

print("REPORT FOR PERFECT DAYS")
print(" ")
print("Total Perfect Days:", perf, "/", len(np.array(df['Close to Next Low'])))
print("% Perfect Days: ", perf/len(np.array(df['Close to Next Low']))*100, "%")
print("--------------------------------------------------------------------------")

print("REPORT FOR MILD PROFIT DAYS")
print(" ")
print("Total MILD PROFIT Days:", len(np.array(df['Close to Next Low'])) - perf - sl, "/", len(np.array(df['Close to Next Low'])))
print("% Perfect Days: ", (len(np.array(df['Close to Next Low'])) - perf - sl)/len(np.array(df['Close to Next Low']))*100, "%")

print(" ")
print("------------------------------END OF ANALYSIS-----------------------------")

REPORT ON LOSS
 
Total SL days: 8 / 16 
 % Day SL hit 50.0 %
--------------------------------------------------------------------------
REPORT FOR PERFECT DAYS
 
Total Perfect Days: 5 / 16
% Perfect Days:  31.25 %
--------------------------------------------------------------------------
REPORT FOR MILD PROFIT DAYS
 
Total MILD PROFIT Days: 3 / 16
% Perfect Days:  18.75 %
 
------------------------------END OF ANALYSIS-----------------------------
